***https://github.com/jeremiedbb/tutorial-euroscipy-2019.git***

**Requirements**
- python > 3.5
- jupyter


- numpy
- scipy


- cython
- numba


- snakeviz


- pandas
- matplotlib


- a C compiler supporting OpenMP
  (for macOS users: ``conda install gcc``)
  
**For conda users**, you can get a ready-to go environment by cloning the repo and running ``conda env create``.
  
<br></br>
<br></br>
<br></br>

In [ ]:
import time
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import numba
from numba import jit, njit, prange

%matplotlib inline
%load_ext cython
%load_ext snakeviz

<br></br>
<br></br>
# Speed up your python code
## An application with pairwise distances

<br></br>
### The problem

Let ``X`` and ``Y`` be two sets of points. For all points in `X`, find its closest point in `Y` w.r.t some given distance ``d``.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall x\in X, c(x) = \underset{y\in Y}{\operatorname{argmin}}{d(x,y)}$

We need to
- compute distances between all points in ``X`` and all points in ``Y``.
  Done using scipy ``cdist``.
- for each point in ``X`` find which point in ``Y`` minimize this distance.
  Done using numpy ``argmin``.

In [ ]:
def cdist_argmin(X, Y, metric):
    distances = cdist(X, Y, metric=metric)
    argmins = np.argmin(distances, axis=1)
    
    return argmins

For example we can use it to draw the Voronoi diagram of the set of points ``Y``.

In [ ]:
def plot(metric, title):
    Xv = np.array([[0.005*(i//200), 0.005*(i%200)] for i in range(40000)])
    Yv = np.random.RandomState(42).random_sample((20, 2))

    indices = cdist_argmin(Xv,Yv, metric=metric)
    plt.scatter(Xv[:,0], Xv[:,1], c=indices, s=10, marker='s')
    plt.scatter(Yv[:,0], Yv[:,1], color='red')
    plt.axis('equal')
    plt.rcParams['figure.figsize'] = [10, 10]
    plt.title(title)

plot('euclidean', 'Euclidean metric')

In [ ]:
plot('cityblock', 'Cityblock metric')

<br></br>
### Efficiency considerations

Let's generate some data for our benchmarks.

In [ ]:
X = np.random.random_sample((10000, 100))
Y = np.random.random_sample((100, 100))

In the ``cdist_argmin`` function, the critical part is the computation of the pairwise distances as shown in the following profiling. If we need to optimize something, this is the part of the code we want to look at.

In [ ]:
%snakeviz cdist_argmin(X, Y, metric='cityblock')

``cdist`` is very efficient when using a predefined metric but very slow when using a custom metric.

In [ ]:
%timeit cdist(X, Y, metric='cityblock')

In [ ]:
def cityblock(x, y):
    v = np.abs(x - y)
    
    return v.sum()

%timeit -r 3 -n 1 cdist(X, Y, metric=cityblock)

The reason is that for a custom metric ``cdist`` relies on generic python code whereas for a predinefined metric it can rely on a specific C implementation.

It's a good candidate to move to compiled code.

<br></br>
### Our custom metric

Through the tutorial we'll use the following metric:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$d(x,y) = \sum_{i=0}^{k}{(i + 1) |x_i - y_i|}$

In [ ]:
def custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1)
    
    return v.dot(w)

In [ ]:
plot(custom_metric, 'Custom metric')

Helpers for accuracy testing and benchmarking

In [ ]:
def test(func, X, Y):
    true_indices = cdist_argmin(X, Y)
    indices = func(X, Y)
    if np.all(true_indices == indices):
        print('correct result \o/')
    else:
        print('incorrect result TT')

In [ ]:
benchs = pd.DataFrame(columns=['version', 'time (s)', 'speedup'])


def bench(func, X, Y, title):
    t = 0
    for i in range(20):
        t_ = time.time()
        func(X, Y)
        t_ = time.time() - t_
        t += t_
        if t_ > 0.5 and i >=3:
            break
    t /= (i + 1)

    i = benchs.shape[0]
    if i == 0:
        speedup = 1
    else:
        speedup = benchs.iloc[0]['time (s)'] / t
    benchs.loc[i] = [title, t, speedup]

    print(benchs)

<br></br>
### Baseline: 

In [ ]:
def cdist_argmin(X, Y):
    distances = cdist(X, Y, metric=custom_metric)
    argmins = np.argmin(distances, axis=1)
    
    return argmins

In [ ]:
test(cdist_argmin, X, Y)

In [ ]:
bench(cdist_argmin, X, Y, 'baseline')

<br></br>
### Cython

#### - Python is valid Cython

In [ ]:
%%cython -f

import numpy as np


def custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1)
    
    return np.dot(v, w)

def cython_cdist_argmin(X, Y):
    distances = np.empty((X.shape[0], Y.shape[0]))
    
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            distances[i, j] = custom_metric(x, y)
    
    return np.argmin(distances, axis=1)

In [ ]:
test(cython_cdist_argmin, X, Y)

In [ ]:
bench(cython_cdist_argmin, X, Y, "cython pure python")

<br></br>
**Cython annotations** (-a option) show which line contains interaction with python objects.

In [ ]:
%%cython -a -f

import numpy as np


def custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1)

    return np.dot(v, w)


def cython_cdist_argmin(X, Y):
    distances = np.empty((X.shape[0], Y.shape[0]))
    
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            distances[i, j] = custom_metric(x, y)
    
    return np.argmin(distances, axis=1)

<br></br>
#### - Get rid of python interactions within the nested loop

Move from python variables to typed variables  
``cdef <type> var = ...``

Python ``range`` keyword is compiled into efficient C code  
e.g. ``for i in range(10)``  

In [ ]:
%%cython -a -f

import numpy as np


def custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1)

    return np.dot(v, w)


def cython_cdist_argmin(X, Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

    distances = np.empty((X.shape[0], Y.shape[0]))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(distances, axis=1)

``distances`` is still a python object (numpy array). We can get an efficient access to its data using a **typed memoryview**.  

In [ ]:
%%cython -a -f

import numpy as np


def custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1)

    return np.dot(v, w)


def cython_cdist_argmin(X, Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

        double[:, :] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(distances, axis=1)

``custom_metric`` is a python function with python arguments implemented in pure python.  
- functions maint to be used in python should be defined using `def`  
- functions maint to be used within cython should be defined using `cdef`, and typed.

We also need to use memoryviews for X and Y.

In [ ]:
%%cython -a -f

import numpy as np


cdef double custom_metric(double[:] x, double[:] y):
    cdef:
        int n = x.shape[0]
        double tmp
        double res = 0.0
        int i

    for i in range(n):
        tmp = x[i] - y[i]
        if tmp < 0:
            tmp = -tmp
        res += (i + 1) * tmp
        
    return res


def cython_cdist_argmin(double[:, :] X, double[:, :] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

        double[:, :] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
test(cython_cdist_argmin, X, Y)

In [ ]:
bench(cython_cdist_argmin, X, Y, "cdef custom_metric")

<br></br>
Cython checks that indexing memoryviews stays within bounds and allows negative indexing. We can disable both using ``boundscheck`` and ``wraparound`` **cython directives**.

In [ ]:
%%cython -a -f
#cython: boundscheck=False, wraparound=False

import numpy as np


cdef double custom_metric(double[:] x, double[:] y):
    cdef:
        int n = x.shape[0]
        double tmp
        double res = 0.0
        int i

    for i in range(n):
        tmp = x[i] - y[i]
        if tmp < 0:
            tmp = -tmp
        res += (i + 1) * tmp
        
    return res


def cython_cdist_argmin(double[:, :] X, double[:, :] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

        double[:, :] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
test(cython_cdist_argmin, X, Y)

In [ ]:
bench(cython_cdist_argmin, X, Y, "cython directives")

<br></br>
#### - Getting the most out of Cython

If we know X and Y will always have the same **memory layout** we can give that information to cython which will produce specialized code.  
Let's assume X and Y are C ordered.

In [ ]:
%%cython -a -f
#cython: boundscheck=False, wraparound=False

import numpy as np


cdef double custom_metric(double[::1] x, double[::1] y):
    cdef:
        int n = x.shape[0]
        double tmp
        double res = 0.0
        int i

    for i in range(n):
        tmp = x[i] - y[i]
        if tmp < 0:
            tmp = -tmp
        res += (i + 1) * tmp
        
    return res


def cython_cdist_argmin(double[:, ::1] X, double[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

        double[:, ::1] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
bench(cython_cdist_argmin, X, Y, "C ordered")

Use functions already implemented in the C standard library: ``fabs``.  
C functions need to be imported with the ``cimport`` keyword.

In [ ]:
%%cython -f
#cython: boundscheck=False, wraparound=False

import numpy as np
from libc.math cimport fabs


cdef double custom_metric(double[::1] x, double[::1] y):
    cdef:
        int n = x.shape[0]
        double res = 0.0
        int i

    for i in range(n):
        res += (i + 1) * fabs(x[i] - y[i])
        
    return res


def cython_cdist_argmin(double[:, ::1] X, double[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

        double[:, ::1] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
test(cython_cdist_argmin, X, Y)

In [ ]:
bench(cython_cdist_argmin, X, Y, "fabs")

GCC can't **vectorize** reduction.
We can either:
- tell GCC that it's fine to reorder operations
- manually unroll the loop

We can specify compiler flags through the ``-c`` option.

In [ ]:
%%cython -c=-Ofast -f
#cython: boundscheck=False, wraparound=False

import numpy as np
from libc.math cimport fabs


cdef double custom_metric(double[:] x, double[:] y):
    cdef:
        int n = x.shape[0]
        double res = 0.0
        int i

    for i in range(n):
        res += (i + 1) * fabs(x[i] - y[i])
        
    return res


def cython_cdist_argmin(double[:, :] X, double[:, :] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int i, j

        double[:, :] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(X[i], Y[j])
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
bench(cython_cdist_argmin, X, Y, "-Ofast")

Manually unroll the loop will be worth only if we use pointers.

In [ ]:
%%cython -f
#cython: boundscheck=False, wraparound=False

import numpy as np
from libc.math cimport fabs


cdef double custom_metric(double* x, double* y, int k):
    cdef:
        double res = 0.0
        int i

    for i in range(k):
        res += (i + 1) * fabs(x[i] - y[i])

    return res


def cython_cdist_argmin(double[:, ::1] X, double[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int k = X.shape[1]
        int i, j

        double[:, ::1] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(&X[i, 0], &Y[j, 0], k)
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
bench(cython_cdist_argmin, X, Y, "pointers")

In [ ]:
%%cython -f
#cython: boundscheck=False, wraparound=False

import numpy as np
from libc.math cimport fabs


cdef double custom_metric(double* x, double* y, int k):
    cdef:
        int rem = k % 4
        double res = 0.0
        int i

    for i in range(0, k-rem, 4):
        res += ((i + 1) * fabs(x[i] - y[i])
               +(i + 2) * fabs(x[i + 1] - y[i + 1])
               +(i + 3) * fabs(x[i + 2] - y[i + 2])
               +(i + 4) * fabs(x[i + 3] - y[i + 3]))
    
    for i in range(i, k):
        res += (i + 1) * fabs(x[i] - y[i])
     
    return res


def cython_cdist_argmin(double[:, ::1] X, double[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int k = X.shape[1]
        int i, j

        double[:, ::1] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(&X[i, 0], &Y[j, 0], k)
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
bench(cython_cdist_argmin, X, Y, "pointers + unrolled loop")

<br></br>
### More cython

#### Profiling cython code

Enabling the ``profile`` cython directive will allow us to profile our code with ``snakeviz``.

In [ ]:
%%cython -f
#cython: boundscheck=False, wraparound=False, profile=True

import numpy as np
from libc.math cimport fabs


cdef double custom_metric(double* x, double* y, int k):
    cdef:
        int rem = k % 4
        double res = 0.0
        int i

    for i in range(0, k, 4):
        res += ((i + 1) * fabs(x[i] - y[i])
               +(i + 2) * fabs(x[i + 1] - y[i + 1])
               +(i + 3) * fabs(x[i + 2] - y[i + 2])
               +(i + 4) * fabs(x[i + 3] - y[i + 3]))
    
    for i in range(i, rem):
        res += (i + 1) * fabs(x[i] - y[i])
     
    return res


def cython_cdist_argmin(double[:, ::1] X, double[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int k = X.shape[1]
        int i, j

        double[:, ::1] distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(&X[i, 0], &Y[j, 0], k)
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
%snakeviz cython_cdist_argmin(X, Y)

<br></br>
#### Generic code with fused types

Our current code is specific to ``double`` dtype (np.float64).

In [ ]:
cython_cdist_argmin(X.astype(np.float32), Y.astype(np.float32))

In [ ]:
%%cython -f
#cython: boundscheck=False, wraparound=False

import numpy as np
from libc.math cimport fabs

ctypedef fused floating:
    float
    double

cdef floating custom_metric(floating* x, floating* y, int k):
    cdef:
        int rem = k % 4
        floating res = 0.0
        int i

    for i in range(0, k, 4):
        res += ((i + 1) * fabs(x[i] - y[i])
               +(i + 2) * fabs(x[i + 1] - y[i + 1])
               +(i + 3) * fabs(x[i + 2] - y[i + 2])
               +(i + 4) * fabs(x[i + 3] - y[i + 3]))
    
    for i in range(i, rem):
        res += (i + 1) * fabs(x[i] - y[i])
     
    return res


def cython_cdist_argmin(floating[:, ::1] X, floating[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int k = X.shape[1]
        int i, j

    dtype = np.float32 if floating is float else np.float64
    cdef floating[:, ::1] distances = np.empty((nx, ny), dtype=dtype)
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = custom_metric(&X[i, 0], &Y[j, 0], k)
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
cython_cdist_argmin(X.astype(np.float32), Y.astype(np.float32))

<br></br>
#### Parallelism with OpenMP

Simply replace the ``range`` keyword by the ``prange`` keyword.

In [ ]:
%%cython -c=-fopenmp -f
#cython: boundscheck=False, wraparound=False

import numpy as np
from libc.math cimport fabs
from cython.parallel cimport prange

ctypedef fused floating:
    float
    double

cdef floating custom_metric(floating* x, floating* y, int k) nogil:
    cdef:
        int rem = k % 4
        floating res = 0.0
        int i

    for i in range(0, k, 4):
        res += ((i + 1) * fabs(x[i] - y[i])
               +(i + 2) * fabs(x[i + 1] - y[i + 1])
               +(i + 3) * fabs(x[i + 2] - y[i + 2])
               +(i + 4) * fabs(x[i + 3] - y[i + 3]))
    
    for i in range(i, rem):
        res += (i + 1) * fabs(x[i] - y[i])
     
    return res


def cython_cdist_argmin(floating[:, ::1] X, floating[:, ::1] Y):
    cdef:
        int nx = X.shape[0]
        int ny = Y.shape[0]
        int k = X.shape[1]
        int i, j

    dtype = np.float32 if floating is float else np.float64
    cdef floating[:, ::1] distances = np.empty((nx, ny), dtype=dtype)
    
    for i in prange(nx, num_threads=2, nogil=True):
        for j in range(ny):
            distances[i, j] = custom_metric(&X[i, 0], &Y[j, 0], k)
    
    return np.argmin(np.asarray(distances), axis=1)

In [ ]:
test(cython_cdist_argmin, X, Y)

In [ ]:
bench(cython_cdist_argmin, X, Y, "prange")

<br></br>
<br></br>
<br></br>
<br></br>
### Numba

Like Cython, Numba can work with Python objects but is much more efficient if it can compile a function without involving the Python interpreter. It's called the **nopython** mode.

To tell Numba which function needs to be compiled, you just have to decorate it: ``@jit``.   
``@jit(nopython=True)`` or ``@njit`` enforces the nopython mode.

In [ ]:
@njit
def f(x, y):
    return x.dot(y)

``f`` is compiled when called for the first time so the first call is longer

In [ ]:
%time f(X[0], Y[0])
%time f(X[0], Y[0])

<br></br>
#### - Type inference

Numba infers the types of the function and its argument at first compilation.

In [ ]:
f.inspect_types()

<br></br>
#### - Numba and Numpy

Numba understands Numpy arrays and supports a lot of Numpy features, meaning using Numpy functions in nopython mode will be compiled into efficient code.

In [ ]:
@njit
def numba_custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1)
    
    return v.dot(w)


@njit
def numba_cdist_argmin(X, Y):
    nx = X.shape[0]
    ny = Y.shape[0]

    distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = numba_custom_metric(X[i], Y[j])
    
    return np.argmin(distances, axis=1)

However this code doesn't compile because:
- Numba support for ``np.dot`` needs to be called with arrays of same dtype
- Numba support for ``np.argmin`` does not handle the ``axis`` parameter.

In [ ]:
@njit
def numba_custom_metric(x, y):
    v = np.abs(x - y)
    w = np.arange(1, x.shape[0] + 1).astype(x.dtype)
    
    return v.dot(w)


@njit
def numba_cdist(X, Y):
    nx = X.shape[0]
    ny = Y.shape[0]

    distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = numba_custom_metric(X[i], Y[j])
    
    return distances

            
def numba_cdist_argmin(X, Y):
    distances = numba_cdist(X, Y)

    return np.argmin(distances, axis=1)

In [ ]:
test(numba_cdist_argmin, X, Y)

In [ ]:
bench(numba_cdist_argmin, X, Y, "naive numba")

<br></br>
#### - Fastmath optimization

Like with cython, telling the compiler that it can reorder operation as it wants can speed things up significantly. This is done through the **fastmath** option.

However we need to rewrite our function in a more C-like way to benefit from it.

In [ ]:
@njit(fastmath=True)
def numba_custom_metric(x, y):
    res = 0

    for i in range(x.shape[0]):
        v = x[i] - y[i]
        if v < 0:
            v = - v
        res += (i + 1) * v
    
    return res


@njit(fastmath=True)
def numba_cdist(X, Y):
    nx = X.shape[0]
    ny = Y.shape[0]

    distances = np.empty((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            distances[i, j] = numba_custom_metric(X[i], Y[j])
    
    return distances

In [ ]:
test(numba_cdist_argmin, X, Y)

In [ ]:
bench(numba_cdist_argmin, X, Y, "numba fastmath")

<br></br>
### More Numba

#### - Profiling

You can profile Numba compiled functions without additional code.

In [ ]:
%snakeviz numba_cdist_argmin(X, Y)

<br></br>
#### - Parallel loop in Numba

Simply replace ``range`` by ``prange`` and enable the ``parallel`` option.
Currently, the only way to specify the number of threads to be use for the ``prange`` is through the ``NUMBA_NUM_THREADS`` environment variable (before imports).

In [ ]:
@njit(parallel=True)
def numba_cdist(X, Y):
    nx = X.shape[0]
    ny = Y.shape[0]

    distances = np.empty((nx, ny))
    
    for i in prange(nx):
        for j in range(ny):
            distances[i, j] = numba_custom_metric(X[i], Y[j])
    
    return distances

In [ ]:
test(numba_cdist_argmin, X, Y)

In [ ]:
bench(numba_cdist_argmin, X, Y, "numba prange")

Numba supports the OpenMP and TBB **threading layers** (and a simple built-in threading layer in case none of those is found).

``pip install tbb`` will install TBB.

In [ ]:
numba.threading_layer()